In [1]:
import json

In [ ]:
with open("data/data-1600-9.json", "r") as f:
  json_data = json.load(f)
len(json_data["testCases"])

In [1]:
import subprocess

In [2]:
for i in range(25, 25 + 16):
  subprocess.run(f"curl https://samate.nist.gov/SARD/api/test-cases/search?language%5B%5D=java&page={i}&limit=100 -o data/data-{i}.json")

In [ ]:
import os
import json
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Create directories if they don't exist
os.makedirs("data_cpp/json", exist_ok=True)

def get_test_cases(page):
    url = f"https://samate.nist.gov/SARD/api/test-cases/search?language%5B%5D=java&state%5B%5D=mixed&page={page}&limit=100"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            # Fix the directory path to match the one we created
            filename = f"data_cpp/json/data-mixed-{page}.json"
            with open(filename, 'w') as f:
                json.dump(data, f, indent=2)
            return f"Downloaded page {page}"
        else:
            return f"Failed to download page {page}: HTTP {response.status_code}"
    except Exception as e:
        return f"Error downloading page {page}: {str(e)}"

# Get first page to check total records
first_page = requests.get("https://samate.nist.gov/SARD/api/test-cases/search?language%5B%5D=java&state%5B%5D=mixed&page=1&limit=100")
total_records = min(first_page.json()['total'], 2000)
print(total_records)
total_pages = (total_records + 99) // 100  # Ceiling division

# Download pages in parallel
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(get_test_cases, page) for page in range(1, total_pages + 1)]
    
    for future in tqdm(as_completed(futures), total=len(futures), desc="Downloading pages"):
        try:
            result = future.result()
            if "Error" in result or "Failed" in result:
                print(f"\n{result}")
        except Exception as e:
            print(f"\nUnexpected error: {str(e)}")

In [ ]:
import os
import json
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from functools import partial
import time
import urllib3

# Increase max retries for the urllib3 connection pool
urllib3.util.retry.Retry.DEFAULT.backoff_factor = 1
urllib3.util.retry.Retry.DEFAULT.total = 5

def download_file(url, download_dir, max_retries=3, retry_delay=2):
    filename = url.split('/')[-1]
    # Add -mixed suffix before .zip extension
    filename_parts = filename.rsplit('.', 1)
    filename = f"{filename_parts[0]}-bad.{filename_parts[1]}"
    
    filepath = os.path.join(download_dir, filename)
    
    # Skip if file already exists and has size > 0
    if os.path.exists(filepath) and os.path.getsize(filepath) > 0:
        return f"Skipped {filename} (already exists)"
    
    for attempt in range(max_retries):
        try:
            # Create a session with custom settings
            session = requests.Session()
            session.mount('https://', requests.adapters.HTTPAdapter(
                max_retries=urllib3.util.Retry(
                    total=5,
                    backoff_factor=1,
                    status_forcelist=[500, 502, 503, 504]
                )
            ))
            
            # Download with increased timeout
            response = session.get(url, stream=True, timeout=30)
            if response.status_code == 200:
                with open(filepath, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                # Verify file was written
                if os.path.exists(filepath) and os.path.getsize(filepath) > 0:
                    return f"Successfully downloaded {filename}"
                else:
                    raise Exception("File was not written correctly")
            else:
                raise requests.exceptions.RequestException(f"HTTP {response.status_code}")
                
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(retry_delay * (attempt + 1))  # Exponential backoff
                continue
            return f"Error downloading {filename} after {max_retries} attempts: {str(e)}"
        finally:
            session.close()
    
    return f"Failed to download {filename} after all retries"

# Get download URLs from JSON files
download_urls = []
json_dir = "data_cpp/json"
for json_file in os.listdir(json_dir):
    if json_file.startswith("data-bad-"):
        with open(os.path.join(json_dir, json_file), 'r') as f:
            data = json.load(f)
            for test_case in data.get('testCases', []):
                download_url = test_case.get('download')
                if download_url:
                    download_urls.append(download_url)

# Create a directory for downloads if it doesn't exist
os.makedirs('data_cpp/zips', exist_ok=True)

# Reduce number of concurrent downloads to avoid overwhelming the connection
max_workers = 20  # Reduced from 50 to 10 for more stability
batch_size = 10    # Reduced batch size

# Create a partial function with the download directory
download_func = partial(download_file, download_dir='data_java1/zips')

# Keep track of failed downloads for potential retry
failed_downloads = []

# Use ThreadPoolExecutor for parallel downloads
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = []
    
    # Process in smaller batches
    for i in range(0, len(download_urls), batch_size):
        batch = download_urls[i:i + batch_size]
        batch_futures = [executor.submit(download_func, url) for url in batch]
        futures.extend(batch_futures)
        
        # Small delay between batches
        time.sleep(1)
    
    # Process completed downloads with progress bar
    with tqdm(total=len(download_urls), desc="Retrying failed downloads") as pbar:
        for future in as_completed(futures):
            try:
                result = future.result()
                if "Error" in result or "Failed" in result:
                    print(f"\n{result}")
                    # Add to failed downloads list for potential future retry
                    failed_downloads.append(result.split()[2])  # Extract filename
            except Exception as e:
                print(f"\nUnexpected error: {str(e)}")
            finally:
                pbar.update(1)

# Print summary
print("\nDownload retry completed")
if failed_downloads:
    print(f"Files that still failed to download ({len(failed_downloads)}):")
    for file in failed_downloads:
        print(f"- {file}")
else:
    print("All files were downloaded successfully!")

In [ ]:
# Unzip files from data/zips to data/unzips using ThreadPoolExecutor
import os
import zipfile
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

def unzip_file(zip_filename):
    """
    Unzip a single zip file
    
    Args:
        zip_filename (str): Name of the zip file to unzip
    """
    zip_path = os.path.join('data_java1', 'zips', zip_filename)
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            # Extract to the destination directory
            zip_ref.extractall(os.path.join('data_java1', 'unzips', zip_filename.split(".zip")[0]))
        return f"Successfully unzipped: {zip_filename}"
    except Exception as e:
        return f"Error unzipping {zip_filename}: {str(e)}"

def main():
    # Create the destination directory if it doesn't exist
    os.makedirs(os.path.join('data_java1', 'unzips'), exist_ok=True)

    # Get list of zip files in the source directory
    zip_files = [f for f in os.listdir(os.path.join('data_java1', 'zips')) if f.endswith('.zip')]

    print(f"Found {len(zip_files)} zip files to process")

    # Use ThreadPoolExecutor for parallel unzipping
    with ThreadPoolExecutor(max_workers=20) as executor:
        # Submit all tasks
        futures = [executor.submit(unzip_file, zip_file) for zip_file in zip_files]
        
        # Process results with progress bar
        with tqdm(total=len(zip_files), desc="Unzipping files") as pbar:
            for future in as_completed(futures):
                try:
                    result = future.result()
                    if "Error" in result:
                        print(f"\n{result}")
                except Exception as e:
                    print(f"\nUnexpected error: {str(e)}")
                finally:
                    pbar.update(1)

    # List the extracted files
    print("\nExtracted files:")
    for extracted_file in os.listdir(os.path.join('data_java1', 'unzips')):
        print(f"- {extracted_file}")

if __name__ == '__main__':
    main()

In [ ]:
import os
for root, dirs, files in os.walk('data_java1/unzips'):
    print(f"Root: {root}")
    print(f"Directories: {dirs}")
    print(f"Files: {len(files)}")
    break

In [ ]:
import shutil
import os

# Create java-src directory if it doesn't exist
os.makedirs('data_cpp/cpp-src', exist_ok=True)

# Walk through the unzips directory
for root, dirs, files in os.walk('data_cpp/unzips'):
    for dir in dirs:
        if dir.endswith("good"):
            for root_sub, _, files in os.walk(os.path.join(root, dir)):
                for file in files:
                    if file.endswith(".cpp") or file.endswith(".c"):
                        source_file_path = os.path.join(root_sub, file)
                        os.makedirs("data_cpp/cpp-src" +"/" + dir, exist_ok=True)
                        dest_file_path = os.path.join("data_cpp/cpp-src", dir, f"{file}")
                        print(dest_file_path)
                        shutil.copy2(source_file_path, dest_file_path)

print("Done")

In [ ]:
import os
import shutil
import json

def extract_mixed_java_files(unzips_dir='data_cpp/unzips', cpp_src_dir='data_cpp/cpp-src'):
    """
    For each folder in unzips_dir ending with 'mixed', parse its manifest.sarif,
    extract the Java file paths from the 'results'->'locations'->'artifactLocation'->'uri' fields,
    and copy those files into java_src_dir/<unzipped_folder>/
    """
    os.makedirs(cpp_src_dir, exist_ok=True)
    for entry in os.listdir(unzips_dir):
        if entry.endswith('mixed'):
            mixed_folder = os.path.join(unzips_dir, entry)
            manifest_path = os.path.join(mixed_folder, 'manifest.sarif')
            if not os.path.isfile(manifest_path):
                print(f"Warning: manifest.sarif not found in {mixed_folder}")
                continue
            try:
                with open(manifest_path, 'r', encoding='utf-8') as f:
                    manifest = json.load(f)
            except Exception as e:
                print(f"Error reading {manifest_path}: {e}")
                continue

            # Defensive: SARIF structure
            runs = manifest.get('runs', [])
            for run in runs:
                results = run.get('results', [])
                for result in results:
                    locations = result.get('locations', [])
                    for loc in locations:
                        artifact_loc = loc.get('physicalLocation', {}).get('artifactLocation', {})
                        uri = artifact_loc.get('uri')
                        if uri and uri.endswith('.cpp'):
                            src_file_path = os.path.join(mixed_folder, uri.replace('/', os.sep))
                            if not os.path.isfile(src_file_path):
                                print(f"Source file not found: {src_file_path}")
                                continue
                            # Create destination directory for this mixed folder
                            dest_dir = os.path.join(cpp_src_dir, entry)
                            os.makedirs(dest_dir, exist_ok=True)
                            dest_file_path = os.path.join(dest_dir, os.path.basename(uri))
                            print(f"Copying {src_file_path} -> {dest_file_path}")
                            shutil.copy2(src_file_path, dest_file_path)
    print("Done")

extract_mixed_java_files()

In [ ]:
import shutil
import os

# Walk through the c-src directory
for root, dirs, files in os.walk('data_c/c-src'):
    for file in files:
        if file.endswith('.cpp'):
            # Get the path relative to c-src directory
            rel_path = os.path.relpath(root, 'data_c/c-src')
            # Get the first folder name after c-src
            parent_folder = rel_path.split(os.sep)[0]
            
            # Source and destination paths
            src_path = os.path.join(root, file)
            dest_path = os.path.join('data_cpp/unzips', parent_folder, file)
            
            # Delete the file
            os.remove(src_path)
            print(f"Deleted {file} from {root}")

# Remove empty directories
for root, dirs, files in os.walk('data_c/c-src', topdown=False):
    for dir in dirs:
        dir_path = os.path.join(root, dir)
        try:
            os.rmdir(dir_path)
            print(f"Removed empty directory: {dir_path}")
        except OSError:
            pass  # Directory not empty

# Try to remove the main c-src directory
try:
    os.rmdir('data_c/c-src')
    print("\nRemoved c-src directory")
except OSError:
    print("\nCould not remove c-src directory (may not be empty)")


# Preprocess data in vul table

In [5]:
import pandas as pd

In [6]:
vul_character_df = pd.read_csv("vuln-char-table-final.csv")
vul_character_df.columns

Index(['Vulnerability Characteristics', 'Total Count', 'Node Type',
       'Example Code'],
      dtype='object')

In [7]:
vul_character_df["Node Type"].unique

<bound method Series.unique of 0                                             CALL
1                                       IDENTIFIER
2     FIELD_IDENTIFIER / fieldAccess / indexAccess
3                                       assignment
4                                         addition
5                                            alloc
6                                      CatchClause
7                                CONTROL_STRUCTURE
8               arrayInitializer / stonesoup_array
9                          logicalAnd / logicalNot
10                                            cast
Name: Node Type, dtype: object>

In [8]:
vul_character_df

,Vulnerability Characteristics,Total Count,Node Type,Example Code
0,Function calls,46035,CALL,int factor = (1 << 31) % random; (1 << 31) % r...
1,Decide the type of the variable,39122,IDENTIFIER,factor; random; Tracer; factor; counter; count...
2,Access a field of an object of aggregate type,13827,FIELD_IDENTIFIER / fieldAccess / indexAccess,vowlessInferentialist; length; splurgeZoograft...
3,Assign values to variables,5559,assignment,int factor = (1 << 31) % random; char counter ...
4,Conduct an arithmetic calculation,2606,addition,e.getClass().getName() + \; e.getClass().getNa...
5,Open or discard a memory space,1307,alloc,new int[size]; new String[stonesoup_value]; ne...
6,Exception handling,1112,CatchClause,catch; catch; catch; catch; catch; catch; catc...
7,Relate to control flow and code structure of t...,1055,CONTROL_STRUCTURE,else; catch; else; catch; else; catch; else; c...
8,Use an array,1037,arrayInitializer / stonesoup_array,<operator>.arrayInitializer; <operator>.arrayI...
9,Conduct a boolean/logical/comparison operation,631,logicalAnd / logicalNot,(stonesoup_counter + stonesoup_offset > 0) && ...


In [9]:
vul_character_df['Node Type'] = vul_character_df['Node Type'].str.strip().str.split(' / ')
vul_character_df

,Vulnerability Characteristics,Total Count,Node Type,Example Code
0,Function calls,46035,[CALL],int factor = (1 << 31) % random; (1 << 31) % r...
1,Decide the type of the variable,39122,[IDENTIFIER],factor; random; Tracer; factor; counter; count...
2,Access a field of an object of aggregate type,13827,"[FIELD_IDENTIFIER, fieldAccess, indexAccess]",vowlessInferentialist; length; splurgeZoograft...
3,Assign values to variables,5559,[assignment],int factor = (1 << 31) % random; char counter ...
4,Conduct an arithmetic calculation,2606,[addition],e.getClass().getName() + \; e.getClass().getNa...
5,Open or discard a memory space,1307,[alloc],new int[size]; new String[stonesoup_value]; ne...
6,Exception handling,1112,[CatchClause],catch; catch; catch; catch; catch; catch; catc...
7,Relate to control flow and code structure of t...,1055,[CONTROL_STRUCTURE],else; catch; else; catch; else; catch; else; c...
8,Use an array,1037,"[arrayInitializer, stonesoup_array]",<operator>.arrayInitializer; <operator>.arrayI...
9,Conduct a boolean/logical/comparison operation,631,"[logicalAnd, logicalNot]",(stonesoup_counter + stonesoup_offset > 0) && ...
